In [ ]:
#@title # <font class="markdown-google-sans">**<font color="red">1.</font>** Install</font>
import os

!git clone https://github.com/jeonchangbin49/De-limiter
print()
!pip install -r "De-limiter/requirements.txt"

cwd = os.getcwd()

In [ ]:
#@title # <font class="markdown-google-sans">**<font color="gray">2.</font>** Drive connection <font size=4>(optional)</font></font>
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

In [ ]:
#@title # <font class="markdown-google-sans">**<font color=lime>3.</font>** Inference</font>
import os
import time
import torch
import tempfile
import soundfile as sf
from pathlib import Path

#-=-=-=-#

Directory_Input = "/content/drive/MyDrive" #@param {type: "string"}
Directory_Output = "/content/drive/MyDrive/_delimited" #@param {type: "string"}

Loudness_Input_LUFS = -18 # @param {"type": "slider", "min": -20, "max": 0, "step": 1}

Avoid_Clipping = False #@param {type: "boolean"}
#@markdown > Forcefully normalize an output by `np.abs(wav).max()` if there exist some values that exceed 0 dBFS

Resample = True #@param {type: "boolean"}
#@markdown > Resample input file to `44100` and to its original sample rate after delimiting.

Use_SingleTrackSet = False #@param {type: "boolean"}
#@markdown > Use SingleTrackSet if input data is too long.

Sort_By = "Name" #@param ["Name", "Size", "Date"]
Sort_Ascendingly = True #@param {type: "boolean"}

Output_Extension = "FLAC" #@param ["WAV", "FLAC"]

Use_GPU = torch.cuda.is_available()

# args
Avoid_Clipping = "" if Avoid_Clipping else "--avoid_clipping"
Use_SingleTrackSet = "--use_singletrackset" if Use_SingleTrackSet else ""
Use_GPU = "true" if Use_GPU else "false"

if Sort_By == "Name":
	Sort_By = (str, not Sort_Ascendingly)
elif Sort_By == "Size":
	Sort_By = (os.path.getsize, Sort_Ascendingly)
elif Sort_By == "Date":
	Sort_By = (os.path.getmtime, not Sort_Ascendingly)

#-=-=-=-#

if not os.path.exists(Directory_Output):
	os.makedirs(Directory_Output)

# validate
inputs = []

for File in Path(Directory_Input).glob("*.*"):
	if not all((
		File.is_file(),
		File.suffix.lower() in (".mp3", ".wav", ".flac")
	)):
		continue

	try:
		sfo = sf.SoundFile(File)

		if Resample:
			output = os.path.join(
				tempfile.gettempdir(),
				"De-limiter",
				os.path.splitext(os.path.basename(File))[0] + ".wav"
			)

			if not os.path.exists(os.path.dirname(output)):
				os.makedirs(os.path.dirname(output))

			!ffmpeg -loglevel 8 -hide_banner \
				-i "$File" \
				-af "asetrate=44100" \
				"$output" \
				-y

			if os.path.exists(output):
				inputs.append({output: sf.SoundFile(File).samplerate})
		else:
			inputs.append({File: sf.SoundFile(File).samplerate})
	except Exception as e:
		print(e)
		pass

if not inputs:
	raise Exception("No input files found in the specified directory")

os.chdir(os.path.join(cwd, "De-limiter"))

def count_iterable(index: int, iterable: iter, fill: str = "0", rounding: int = 2) -> tuple:
	liter = len(iterable)
	citer = str(index).rjust(len(str(liter)), fill)
	perc = f"{100 * index / liter:.{rounding}f}"
	return (citer, str(liter), perc)

inputs.sort(key = Sort_By[0], reverse = Sort_By[1])

for i, File in enumerate(inputs, start = 1):
	file = list(File.keys())[0]
	rate = list(File.values())[0]

	# snapshot BEFORE inference
	before = {p.name for p in Path(Directory_Output).iterdir() if p.is_file()}

	print("[{0}/{1}] ({2}%)".format(*count_iterable(i, inputs)))

	!python inference.py \
		--data_root "$file" \
		--output_directory "$Directory_Output" \
		--loudnorm $Loudness_Input_LUFS \
		$Avoid_Clipping \
		$Use_GPU \
		$Use_SingleTrackSet

	# detect new file
	after = list(Path(Directory_Output).iterdir())
	new_files = [p for p in after if p.is_file() and p.name not in before]

	if not new_files:
		raise FileNotFoundError("Inference produced no new output file")

	final = max(new_files, key = lambda p: p.stat().st_mtime)

	# strip ALL extensions
	base = final.stem
	while "." in base:
		base = Path(base).stem

	# final output name
	final_out = final.with_name(
		f"{base}_delimited_{Loudness_Input_LUFS}.{Output_Extension.lower()}"
	)

	# restore tags
	tmp = final_out.with_name(final_out.stem + "_tmp." + Output_Extension.lower())
	resample_str = f'-af "asetrate={rate}"' if Resample else ""

	!ffmpeg -loglevel 8 -hide_banner \
		-i "$final" -i "$file" \
		$resample_str \
		-map 0:a -map_metadata 1 \
		-compression_level 8 "$tmp" \
		-y

	final.unlink()
	tmp.rename(final_out)

	if inputs.index(File) != len(inputs):
		print()

os.chdir(cwd)